In [4]:
import numpy as np
import trimesh
import scipy

In [6]:
# create test and train set 
# save it as txt files
import glob
import random
def get_random_train_list():
    files = glob.glob('data/xyz/*.xyz')
    train = random.sample(files, 120)
    train_set = set(train)
    train_list = []
    test_list = []
    for file in files:
        name = file.split('/')[-1].split('.')[0]
        if file in train_set:
            train_list.append(name)
        else:
            test_list.append(name)

    return train_list, test_list
def write_list_to_file(filename,my_list):
    with open(filename, 'w') as f:
        for item in my_list:
            f.write("%s\n" % item)
train_list, test_list = get_random_train_list()
write_list_to_file('data/trainingset.txt', train_list)
write_list_to_file('data/testset.txt', test_list)

In [6]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import coo_matrix, eye


from trimesh import triangles


def filter_humphrey(mesh,
                    alpha=0.1,
                    beta=0.5,
                    iterations=10,
                    laplacian_operator=None):
    """
    Smooth a mesh in-place using laplacian smoothing
    and Humphrey filtering.
    Articles
    "Improved Laplacian Smoothing of Noisy Surface Meshes"
    J. Vollmer, R. Mencl, and H. Muller
    Parameters
    ------------
    mesh : trimesh.Trimesh
      Mesh to be smoothed in place
    alpha : float
      Controls shrinkage, range is 0.0 - 1.0
      If 0.0, not considered
      If 1.0, no smoothing
    beta : float
      Controls how aggressive smoothing is
      If 0.0, no smoothing
      If 1.0, full aggressiveness
    iterations : int
      Number of passes to run filter
    laplacian_operator : None or scipy.sparse.coo.coo_matrix
      Sparse matrix laplacian operator
      Will be autogenerated if None
    """
    # if the laplacian operator was not passed create it here
    if laplacian_operator is None:
        laplacian_operator = laplacian_calculation(mesh)

    # get mesh vertices as vanilla numpy array
    vertices = mesh.vertices.copy().view(np.ndarray)
    # save original unmodified vertices
    original = vertices.copy()

    # run through iterations of filter
    for _index in range(iterations):
        vert_q = vertices.copy()
        vertices = laplacian_operator.dot(vertices)
        vert_b = vertices - (alpha * original + (1.0 - alpha) * vert_q)
        vertices -= (beta * vert_b + (1.0 - beta) *
                     laplacian_operator.dot(vert_b))

    # assign modified vertices back to mesh
    mesh.vertices = vertices
    return mesh, laplacian_operator
def laplacian_calculation(mesh, equal_weight=True):
    """
    Calculate a sparse matrix for laplacian operations.
    Parameters
    -------------
    mesh : trimesh.Trimesh
      Input geometry
    equal_weight : bool
      If True, all neighbors will be considered equally
      If False, all neightbors will be weighted by inverse distance
    Returns
    ----------
    laplacian : scipy.sparse.coo.coo_matrix
      Laplacian operator
    """
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors

    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)


    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    if equal_weight:
        # equal weights for each neighbor
        data = np.concatenate([[1.0 / len(n)] * len(n)
                               for n in neighbors])
    else:
        # umbrella weights, distance-weighted
        # use dot product of ones to replace array.sum(axis=1)
        ones = np.ones(3)
        # the distance from verticesex to neighbors
        norms = [1.0 / np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                 for i, n in enumerate(neighbors)]
        # normalize group and stack into single array
        data = np.concatenate([i / i.sum() for i in norms])

    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    
    # return matrix,data
    return matrix

# mesh = trimesh.load_mesh('../2plane.obj')
# matrix,data = laplacian_calculation(mesh,equal_weight=False)
# print('laplacian matrix: ', matrix.shape,'\n',matrix)

In [5]:
from random import sample
import glob
import trimesh
def save_xyz(pts, file_name):
    # print(pts)
    s = trimesh.util.array_to_string(pts)
    with open(file_name, 'w') as f:
        f.write("%s\n" % s)
def sample_k(matrix,k):
    # matrix = matrix.toarray()
    result = []
    for i,row in enumerate(matrix):
        row = list(row)
        if len(row) >= k:
            k_nearest = sample(row,k)
        else:
            while len(row) < k:
                row.append(0.0)
            k_nearest = row
        result.append(k_nearest)
    return result
def save_vertices():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        vertices = mesh.vertices
        dest_name = 'data/xyz/' +file.split('/')[-1].split('.')[0] +'.xyz'
        save_xyz(vertices, dest_name)
        
def create_target():
    files = glob.glob('data/noisy/*.obj')
    for file in files:
        print(file)
        mesh = trimesh.load_mesh(file)
        matrix = laplacian_calculation(mesh, equal_weight=False)
        result = sample_k(matrix, 5)
    
        dest_name = 'data/laplacian/' +file.split('/')[-1].split('.')[0] +'.laplacian'
        save_xyz(result, dest_name)

save_vertices()


data/noisy/meshes_march2_mesh_0120.obj
data/noisy/meshes_squat2_mesh_0140.obj
data/noisy/meshes_handstand_mesh_0020.obj
data/noisy/meshes_jumping_mesh_0080.obj
data/noisy/meshes_march2_mesh_0170.obj
data/noisy/meshes_march2_mesh_0020.obj
data/noisy/meshes_handstand_mesh_0060.obj
data/noisy/meshes_bouncing_mesh_0120.obj
data/noisy/meshes_squat2_mesh_0020.obj
data/noisy/meshes_march2_mesh_0080.obj
data/noisy/meshes_handstand_mesh_0170.obj
data/noisy/meshes_squat1_mesh_0000.obj
data/noisy/meshes_march1_mesh_0100.obj
data/noisy/meshes_march2_mesh_0150.obj
data/noisy/meshes_march2_mesh_0140.obj
data/noisy/meshes_march1_mesh_0030.obj
data/noisy/meshes_jumping_mesh_0020.obj
data/noisy/meshes_march1_mesh_0070.obj
data/noisy/meshes_crane_mesh_0150.obj
data/noisy/meshes_march2_mesh_0230.obj
data/noisy/meshes_squat2_mesh_0110.obj
data/noisy/meshes_march1_mesh_0050.obj
data/noisy/meshes_crane_mesh_0020.obj
data/noisy/meshes_jumping_mesh_0050.obj
data/noisy/meshes_squat1_mesh_0070.obj
data/noisy/me

In [81]:

def make_smooth(mesh, matrix= None):
    mesh = trimesh.load_mesh(mesh)
    if matrix != None:
        matrix = matrix
    else:
        print("")
        matrix = trimesh.smoothing.laplacian_calculation(mesh, equal_weight= True)
        
    smoothed_mesh = trimesh.smoothing.filter_taubin(mesh,
                    lamb=0.4,
                    nu=0.5,
                    iterations=10,
                    laplacian_operator=matrix)
    return smoothed_mesh
mesh = trimesh.load_mesh('data/noisy/meshes_bouncing_mesh_0020.obj')
matrix_2 = laplacian_cotangent_knn(mesh,3)
smoothed_mesh =  make_smooth('data/noisy/meshes_bouncing_mesh_0020.obj', matrix_2)
output = smoothed_mesh.export('output/meshes_bouncing_mesh_0020_smooth_knn1.obj')
mesh = trimesh.load_mesh('output/meshes_bouncing_mesh_0020_smooth_knn1.obj')
mesh.show()
# print('=============================\n', matrix)


/tmp/ipykernel_179293/3843965187.py:114: RuntimeWarning: invalid value encountered in true_divide
  data = np.concatenate([i / np.array(i).sum() for i in data])


In [33]:
import torch
import numpy as np
from sklearn.neighbors import NearestNeighbors
from chamferdist import ChamferDistance
def calculate_loss(mesh1, mesh2):
    loss = ChamferDistance()
    mse_loss = torch.nn.MSELoss()
    mesh1 = trimesh.load_mesh(mesh1)
    mesh2 = trimesh.load_mesh(mesh2)
    v1, v2 = mesh1.vertices.view(np.ndarray),mesh2.vertices.view(np.ndarray)
    v1, v2  = torch.tensor(v1).float().unsqueeze(0), torch.tensor(v2).float().unsqueeze(0)
    
    chamfer_loss = loss(v1, v2)
    mse_loss = mse_loss(v1, v2)
    return chamfer_loss, mse_loss

mesh1 = 'data/smooth/meshes_bouncing_mesh_0020.obj'
mesh2 = 'data/noisy/meshes_bouncing_mesh_0020.obj'

chamfer, mse = calculate_loss(mesh1, mesh2)
print('===================================')
print('loss(smooth_gt,noisy_gt)')
print('ChamferDistance: ', chamfer)
print('MSE: ', mse)
print('===================================')
'''
===================================
between Equal weight False & True: loss(smooth_gt,smooth_true)
ChamferDistance:  tensor(0.0306)
MSE:  tensor(0.1742)

===================================

loss(smooth_gt,smooth_pred_true)
ChamferDistance:  tensor(0.0948)
MSE:  tensor(0.1706)
===================================

===================================
loss(smooth_gt,smooth_pred_false)
ChamferDistance:  tensor(0.0399)
MSE:  tensor(0.1719)
===================================

===================================
loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0296)
MSE:  tensor(0.1707)
===================================
'''

loss(smooth_gt,noisy_gt)
ChamferDistance:  tensor(0.0296)
MSE:  tensor(0.1707)


'\n===================================\nbetween Equal weight False & True: loss(smooth_gt,smooth_true)\nChamferDistance:  tensor(0.0306)\nMSE:  tensor(0.1742)\n\n===================================\n\nloss(smooth_gt,smooth_pred_true)\nChamferDistance:  tensor(0.0948)\nMSE:  tensor(0.1706)\n===================================\n\n===================================\nloss(smooth_gt,smooth_pred_false)\nChamferDistance:  tensor(0.0399)\nMSE:  tensor(0.1719)\n===================================\n'

In [234]:
# Replace the Trimesh Laplacian with the cotangent Laplacian
# Have a function that computes the cotangent weights, given a center point and its k neighbors
# Compute k nearest neighbors
# For each of these k nearest neighbors,
# check if there is an edge in the between the center point and this neighbor. If not, return 0
# If yes, return cotagent weight

# Aijk = np.sqrt(s(s−lik)(s−lkj)(s−lij))
# s = (lik +lkj +lij)/2

# wij = lij + lik + ljk/(8*Aijk) + lij + lih + ljh/(8*Aijh)
import heapq
import numpy as np

get_distance = lambda p1, p2: np.sqrt(np.sum((p1 - p2) ** 2, axis=0))
def get_k_nearest_neighbors(vertices):
    result = {}
    for i,p1 in enumerate(vertices):
        h = []
        for j, p2 in enumerate(vertices):
            if i == j:
                continue
            dist = (get_distance(p1,p2))
            heapq.heappush(h,(float(dist), j))
        k_nearest = []
        for k in range(4): 
            k_nearest.append(heapq.heappop(h))
        result[i] = k_nearest
    return result

def laplacian_cotangent_knn(mesh, k_nearest):
  
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors

    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    # print('row: ',row)
    # print('col: ', col)

    data = []
    # data = np.concatenate([[1.0 / len(n)] * len(n)
    #                            for n in neighbors])
    for i in k_nearest:
        neighbor = set(neighbors[i])
        temp = [0] * len(neighbor)
        # print(temp)
        for nearest in k_nearest[i]:
            lij, j = nearest
            #check if j exist in neighbor
            if j not in neighbor:
                continue
            k = None
            h = None
            # print(nearest)
            # print(neighbors[j])
            for third_point in neighbors[j]:
                if third_point in neighbor and third_point != i:
                    k = third_point
                    for fourth_point in neighbors[k]:
                        if h is not None:
                            break
                        if fourth_point == i or fourth_point ==j:
                            continue
                        for fifth_point in neighbors[fourth_point]:
                            if fifth_point == i:
                                h = fourth_point
                                break
            # wij = lij + lik + ljk/(8*Aijk) + lij + lih + ljh/(8*Aijh)
            # Aijk = np.sqrt(s(s−lik)(s−lkj)(s−lij))
            # s = (lik +lkj +lij)/2
            pi,pj, pk = vertices[i],vertices[j],vertices[k]
            lik, lkj = get_distance(pi,pk),get_distance(pk,pj)
            s = (lik +lkj +lij)/2

            #----------Aijk--------------
            Aijk = np.sqrt(s* (s - lik) * (s - lkj) * (s - lij))

            #---------------------------
            pi,pj, ph = vertices[i],vertices[j],vertices[h]
            lih, ljh = get_distance(pi,ph),get_distance(pj,ph)
            s = (lih+ ljh +lij)/2

            #----------Aijh--------------
            Aijh = np.sqrt(s* (s - lih) * (s - ljh) * (s - lij))
            #---------- wij --------------
            wij = lij + lik + lkj/(8*Aijk) + lij + lih + ljh/(8*Aijh)

            #find index for the wij
            index = (neighbors[i]).index(j)

            temp[index] = wij
            print(wij)
        data.extend(temp)
        



    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix
def laplacian_cotangent(mesh):
       # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors

    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])

    # print('row: ',row)
    # print('col: ', col)

    data = []
    # data = np.concatenate([[1.0 / len(n)] * len(n)
    #                            for n in neighbors])
    for i in k_nearest:
        neighbor = set(neighbors[i])
        temp = [0] * len(neighbor)
        # print(temp)
        for nearest in k_nearest[i]:
            lij, j = nearest
            #check if j exist in neighbor
            if j not in neighbor:
                continue
            k = None
            h = None
            # print(nearest)
            # print(neighbors[j])
            for third_point in neighbors[j]:
                if third_point in neighbor and third_point != i:
                    k = third_point
                    for fourth_point in neighbors[k]:
                        if h is not None:
                            break
                        if fourth_point == i or fourth_point ==j:
                            continue
                        for fifth_point in neighbors[fourth_point]:
                            if fifth_point == i:
                                h = fourth_point
                                break
            # wij = lij + lik + ljk/(8*Aijk) + lij + lih + ljh/(8*Aijh)
            # Aijk = np.sqrt(s(s−lik)(s−lkj)(s−lij))
            # s = (lik +lkj +lij)/2
            pi,pj, pk = vertices[i],vertices[j],vertices[k]
            lik, lkj = get_distance(pi,pk),get_distance(pk,pj)
            s = (lik +lkj +lij)/2

            #----------Aijk--------------
            Aijk = np.sqrt(s* (s - lik) * (s - lkj) * (s - lij))

            #---------------------------
            pi,pj, ph = vertices[i],vertices[j],vertices[h]
            lih, ljh = get_distance(pi,ph),get_distance(pj,ph)
            s = (lih+ ljh +lij)/2

            #----------Aijh--------------
            Aijh = np.sqrt(s* (s - lih) * (s - ljh) * (s - lij))
            #---------- wij --------------
            wij = lij + lik + lkj/(8*Aijk) + lij + lih + ljh/(8*Aijh)

            #find index for the wij
            index = (neighbors[i]).index(j)

            temp[index] = wij
        data.extend(temp)
        



    # create the sparse matrix
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix
mesh = trimesh.load_mesh('../2plane.obj')
vertices = mesh.vertices
k_nearest = get_k_nearest_neighbors(vertices)
        
matrix = laplacian_cotangent(mesh)
# print(matrix)

# k_nearest



    


In [2]:
def laplacian_calculation(mesh, equal_weight=True):
    """
    Calculate a sparse matrix for laplacian operations.
    Parameters
    -------------
    mesh : trimesh.Trimesh
      Input geometry
    equal_weight : bool
      If True, all neighbors will be considered equally
      If False, all neightbors will be weighted by inverse distance
    Returns
    ----------
    laplacian : scipy.sparse.coo.coo_matrix
      Laplacian operator
    """
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors
    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    # stack neighbors to 1D arrays
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])


    ones = np.ones(3)
    # the distance from verticesex to neighbors
    norms = [1.0 / np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    # normalize group and stack into single array
    # data = np.concatenate([i / i.sum() for i in norms])
    data = [i / i.sum() for i in norms]

    # create the sparse matrix
    # matrix = coo_matrix((data, (row, col)),
    #                     shape=[len(vertices)] * 2)

    return data
    



In [26]:
mesh = trimesh.load_mesh('data/noisy/meshes_crane_mesh_0020.obj')
# neighbors = mesh.vertex_neighbors
# vertices = mesh.vertices.view(np.ndarray)
# norms = [1.0/ np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
#                 for i, n in enumerate(neighbors)]
data = laplacian_calculation(mesh)
print(data)
# for weight in data:
#     print(ty)
#     break


In [60]:

from scipy.sparse import coo_matrix, eye
import numpy as np
from scipy.spatial import distance

def laplacian_knn(mesh, k):
    # get the vertex neighbors from the cache
    neighbors = mesh.vertex_neighbors
    # avoid hitting crc checks in loops
    vertices = mesh.vertices.view(np.ndarray)

    
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]

    # stack neighbors to 1D arrays
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])

    ones = np.ones(3)
    # the distance from verticesex to neighbors
    norms = [1.0 / np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    # normalize group and stack into single array
    # data = np.concatenate([i / i.sum() for i in norms])
    data = [i / i.sum() for i in norms]
    new_data = []
    for i, knn in enumerate( closest):
        temp = [0.0] * len(knn) 
        for idx, value in enumerate(knn):
            if value in neighbors[i]:
                index = list.index(neighbors[i],value)
                temp[idx] = data[i][index]

        new_data.append(temp)

    new_data = np.concatenate(new_data)
    # create the sparse matrix
    matrix = coo_matrix((new_data, (row, col)),
                        shape=[len(vertices)] * 2)

    return matrix
# matrix = laplacian_knn(mesh,5)


In [76]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import coo_matrix, eye

def get_cotangent_weight(i,neighbor,neighbors,norms, KNN):
    def get_k_h(i,j):
        list1, list2 = neighbors[i], neighbors[j]
        #if there is morethan 2 intersection choose the first 2 points
        intersection = list(set(list1).intersection(list2))
        try:
            k,h = intersection
        except:
            k,h = intersection[:2]
            # print(intersection)
            # print(i,j)
            # print('k,h', )

        return k,h
    def get_distance(i,j,k,h):
        j_idx = neighbors[i].index(j)
        lij = norms[i][j_idx]

        k_idx = neighbors[j].index(k)
        ljk = norms[j][k_idx]

        i_idx = neighbors[k].index(i)
        lki = norms[k][i_idx]

        h_idx = neighbors[j].index(h)
        ljh = norms[j][h_idx]

        i_idx = neighbors[h].index(i)
        lhi = norms[h][i_idx]

        return lij, ljk, lki, ljh, lhi 

    result = []
    for j in KNN:
        #if j in neighbor find Cotangent weight else assign 0
        wij = 0
        if j in neighbor:
        #find k * h
            k,h  = get_k_h(i,j)
            lij, ljk, lki, ljh, lhi = get_distance(i,j,k,h)

            s_ijk = (lij + ljk + lki)/2
            A_ijk = 8 *  np.sqrt(s_ijk * ( s_ijk - lij) * ( s_ijk- ljk) * ( s_ijk - lki))

            s_ijh = (lij + ljh + lhi)/2
            A_ijh = 8 *  np.sqrt(s_ijh * ( s_ijh - lij) * ( s_ijh- ljh) * ( s_ijh - lhi))

            
            wij = ((-lij**2 + ljk ** 2 + lki**2)/  A_ijk)  + ((-lij**2 + ljh ** 2 + lhi**2)/  A_ijh)
            if wij == np.nan:
                print('here')
        
        result.append(wij)
        
    return result
def laplacian_cot(mesh):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    # norms = [i / i.sum() for i in norms]
    data = []

    for i, neighbor in enumerate(neighbors):
        weight = get_cotangent_weight(i,neighbor, neighbors,norms, neighbor)
        data.append(weight)
        # create the sparse matrix
    
    col = np.concatenate(neighbors)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(neighbors)])
    
    # data = np.array(data)
    data = np.concatenate([i / np.array(i).sum() for i in data])
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix

def laplacian_cotangent_knn(mesh, k):
    neighbors = mesh.vertex_neighbors
    
    vertices = mesh.vertices.view(np.ndarray)

    ones = np.ones(3)
    norms = [np.sqrt(np.dot((vertices[i] - vertices[n]) ** 2, ones))
                for i, n in enumerate(neighbors)]
    D = distance.squareform(distance.pdist(vertices))
    closest = np.argsort(D, axis=1)
    closest = closest[:, 1:k+1]



    # norms = [i / i.sum() for i in norms]
    data = []

    for i, KNN in enumerate(closest):
        neighbor = neighbors[i]
        weight = get_cotangent_weight(i,neighbor, neighbors,norms, KNN)
        data.append(weight)
        # create the sparse matrix
    
    col = np.concatenate(closest)
    row = np.concatenate([[i] * len(n)
                          for i, n in enumerate(closest)])
    
    # data = np.array(data)
    data = np.concatenate([i / np.array(i).sum() for i in data])
    
    matrix = coo_matrix((data, (row, col)),
                        shape=[len(vertices)] * 2)
    return matrix

mesh = trimesh.load_mesh('data/noisy/meshes_bouncing_mesh_0020.obj')
matrix_1 = laplacian_cotangent_knn(mesh, 10)
# print(matrix)